# Generate stores & customers for the demo

## Setup the environment

### Install packages

> **Note:** After running the pip command you should restart the Jupyter kernel.<br>
> To restart the kernel, click on the kernel-restart button in the notebook's action toolbar (the refresh icon next to the **Code** button).

In [1]:
!pip install faker
!pip install pymorton
!pip install v3io
!pip install v3io_frames

### imports

In [2]:
import logging
import os
import pickle
import random
import itertools

# Demo
import pandas as pd
import pymorton as pm
from faker import Faker
from faker.providers import BaseProvider

# DB
import v3io as v3c
import v3io_frames as v3f

### Define environment variables

In [3]:
# Stores
NUMBER_OF_STORES=50
STORES_TABLE=os.path.join(os.getenv('V3IO_USERNAME', 'iguazio'), 'stores')
ACCURACY=20
BASE_ACCURACY=14

# Customers
NUMBER_OF_CUSTOMERS=5000
CUSTOMERS_TABLE=os.path.join(os.getenv('V3IO_USERNAME', 'iguazio'), 'customers')
os.environ['CUSTOMERS_DATA_FILE_DIR'] = os.path.join(os.getcwd(), 'data')
os.environ['CUSTOMERS_DATA_FILE_PATH'] = os.path.join(os.environ['CUSTOMERS_DATA_FILE_DIR'], 'data_train_n.csv')
!mkdir -p ${CUSTOMERS_DATA_FILE_DIR}

In [3]:
!curl -L "iguazio-sample-data.s3.amazonaws.com/data_train_n.csv" > ${CUSTOMERS_DATA_FILE_PATH}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  399M  100  399M    0     0  16.4M      0  0:00:24  0:00:24 --:--:-- 15.5M


In [4]:
!ls -lah ${CUSTOMERS_DATA_FILE_DIR}

total 0
-rw-r--r-- 1 50 nogroup 400M Aug  4 12:09 data_train_n.csv


### Open DB client

In [5]:
client = v3f.Client('framesd:8081', container='users')

## Generate Stores

### Location provider

In [6]:
class LocationProvider(BaseProvider):
    '''
    Creates locations within base_location

    Uses QuadTree for Geohashing
        @{http://tech.taskrabbit.com/blog/2015/06/09/elasticsearch-geohash-vs-geotree/}
        @{http://mapzen.github.io/leaflet-spatial-prefix-tree/}
        @{http://blog.notdot.net/2009/11/Damn-Cool-Algorithms-Spatial-indexing-with-Quadtrees-and-Hilbert-Curves}
    '''    
    def location(self, location_base: str, base_acc=10, acc=20):
        coordinates = location_base[:base_acc]
        for i in range(acc-len(coordinates)):
            coordinates += str(random.randint(0, 3))
        return coordinates

Add the location provider as a faker provider

In [7]:
faker = Faker()
faker.add_provider(LocationProvider)

### Define scenario environment (locations)

In [8]:
london_city = (51.514926, -0.089580)
london_city_south = (51.501593, -0.094942)
london_west = (51.512309, -0.128966)
london_south_west = (51.495022, -0.162268)

london_coordinates = []
london_coordinates.append(london_city)
london_coordinates.append(london_city_south)
london_coordinates.append(london_west)
london_coordinates.append(london_south_west)

london_coordinates_qt = list(map(lambda cooridnate: pm.interleave_latlng(*cooridnate), london_coordinates))
london_coordinates_qt

['03311311313011311011000321002320',
 '03311311311233323013031101320003',
 '03311311313010023000032330133111',
 '03311311311222300331010333220231']

### Define Store class

In [9]:
class Store(object):
    def __init__(self, store_id: int, name: str, location: str):
        self.store_id = store_id if store_id is not None else random.randint(1, int(os.genev('NUMBER_OF_STORES', 500)))
        self.location = location
        (self.lat, self.long) = pm.deinterleave_latlng(location)
        self.store_name = name
        self.customers = 0
    
    def json(self):
        json_store = {
            'id': self.store_id,
            'name': self.store_name,
            'location': self.location,
            'latitude': self.lat,
            'longitude': self.long,
            'customers': self.customers
        }
        return json_store

### Create stores

In [10]:
def create_stores(faker, number_of_stores: int=500):
    remaining = number_of_stores
    stores = []
    while remaining > 0:
        # Create stores
        stores_batch = [Store(faker.msisdn(), faker.company(), faker.location(random.choice(london_coordinates_qt), BASE_ACCURACY, ACCURACY)).json() 
              for i in range(remaining)]
        
        # Count for duplicate locations
        locations = list(map(lambda store: store['location'], stores_batch))
        locations = list(dict.fromkeys(locations))
        remaining -= len(locations)
        
        # Update stores list
        stores.append(stores_batch)

    stores = list(itertools.chain.from_iterable(stores))
    stores = pd.DataFrame.from_records(stores)
    stores = stores.set_index(['location'])
    stores = stores[~stores.index.duplicated(keep='first')]
    return stores

In [11]:
client.delete('kv', STORES_TABLE, if_missing=1)

In [12]:
stores = create_stores(faker, NUMBER_OF_STORES)
stores.head(5)

,customers,id,latitude,longitude,name
location,,,,,
03311311311222120103,0,7044947204572,51.487770,-0.163078,Bishop-Cole
03311311313010223333,0,7317394796707,51.525536,-0.126686,Bailey LLC
03311311313010313113,0,8712626224402,51.517982,-0.110207,Ryan-Hopkins
03311311311222132202,0,6521901547120,51.491890,-0.159302,Sullivan-Brown
03311311313010023003,0,8883522350757,51.512489,-0.128746,Walker-Harper


### Upload to DB 

In [13]:
# client.delete('kv', STORES_TABLE)
client.write('kv', STORES_TABLE, stores)

In [14]:
client.read('kv', STORES_TABLE).head(5)

,customers,id,latitude,longitude,name
location,,,,,
03311311313010221120,0,6040149292924,51.521072,-0.127716,Collins-Compton
03311311311233232223,0,1388437311094,51.503563,-0.104027,"Bush, Campbell and Crawford"
03311311313011111303,0,7393751159414,51.505623,-0.088921,Kirby-Romero
03311311311222130330,0,3332001482053,51.489487,-0.157242,Little and Sons
03311311311222211100,0,2516602914090,51.492920,-0.166168,"West, Brooks and Wilson"


## Generate customers

In [15]:
class Customer(object):
    def __init__(self, id=0, location='0', products=""):
        self.id = id
        self.location = location
        self.products = products
        
    def json(self):
        json_customer = {
            'id': self.id,
            'location': self.location,
            'products': self.products
        }
        return json_customer

In [19]:
def create_customers(customers_data: pd.DataFrame, number_of_customers: int):
    available_customer = list(customers_data.user_id)
    customers = [Customer(id=i,
                          products=customers_data.loc[customers_data['user_id'] == random.choice(available_customer)]\
                                    .drop(['user_id', 'Unnamed: 0'], axis=1)\
                                    .to_json()).json() for i in range(number_of_customers)]
    customers = pd.DataFrame.from_records(customers)
    customers.set_index(['id'])
    
    # Verify string length for KV
    customers['verification'] = customers['products'].apply(lambda x: len(x.encode('utf-8')))
    customers = customers.loc[customers['verification'] <= 61000]
    customers = customers.drop(columns='verification')
    
    # Fix index
    customers = customers.reset_index(drop=True)
    return customers

In [20]:
# Load customers seed data
customers_data = pd.read_csv(os.environ['CUSTOMERS_DATA_FILE_PATH'])

# Create actual customers
customers = create_customers(customers_data, NUMBER_OF_CUSTOMERS)
customers.head(5)

,id,location,products
0,0,0,"{""user_product_reordered_ratio"":{""11355"":1.0,""..."
1,1,0,"{""user_product_reordered_ratio"":{""277139"":1.0,..."
2,2,0,"{""user_product_reordered_ratio"":{""5952"":1.0,""2..."
3,3,0,"{""user_product_reordered_ratio"":{""41610"":1.0,""..."
4,4,0,"{""user_product_reordered_ratio"":{""170321"":1.0,..."


## Upload to DB

In [21]:
client.delete('kv', CUSTOMERS_TABLE, if_missing=1)
client.write('kv', CUSTOMERS_TABLE, dfs=customers)

In [22]:
print(f'Customers table sized: {client.read("kv", CUSTOMERS_TABLE).shape}')
client.read('kv', CUSTOMERS_TABLE).head(5)

Customers table sized: (4981, 3)


,id,location,products
idx,,,
461,463,0,"{""user_product_reordered_ratio"":{""294423"":1.0,..."
630,633,0,"{""user_product_reordered_ratio"":{""91075"":1.0,""..."
917,922,0,"{""user_product_reordered_ratio"":{""6898"":1.0,""1..."
1137,1143,0,"{""user_product_reordered_ratio"":{""240572"":1.0,..."
2976,2988,0,"{""user_product_reordered_ratio"":{""53855"":1.0,""..."


## Generate predictions

In [27]:
# client.delete('tsdb', 'predictions', if_missing=1)
client.create('tsdb', 'predictions', rate='1/s', if_exists=1)